In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

# Reading in the Data

In [2]:
%cd ~/Desktop/CoverageNet/src/03_coverageNet/02_score_attempt

/Users/James/Desktop/CoverageNet/src/03_coverageNet/02_score_attempt


In [3]:
# reading in the classification model
model_classification = load_model('pass_attempt_classification_model')

In [4]:
import tensorflow.keras.backend as K
def crps(y_true, y_pred):
    y_pred =  K.cumsum(y_pred, axis=1)
    ym =  K.cast(K.reshape(K.argmax(y_true, axis=1) - 99, (-1, 1)), 
        dtype='int32')
    n = K.arange(-99, 100)
    step = K.cast(K.greater_equal(n - ym, 0), dtype='float32')
    return K.mean(K.sum(K.square(y_pred - step), axis=1)) / 199

In [5]:
model_caught_yards = load_model('pass_caught_yards_model',
                               custom_objects={'crps':crps})

In [6]:
model_interception_yards = load_model('interceptions_yards_model',
                               custom_objects={'crps':crps})

In [ ]:
# reading in the intermediate data
pass_arrived_score = pd.read_csv("~/Desktop/CoverageNet/src/03_coverageNet/03_score_tracking/intermediates/intermediate_tensor_score.csv")

In [9]:
pass_arrived_score = pd.read_csv("~/Desktop/CoverageNet/src/03_coverageNet/02_score_attempt/outputs/pass_attempt_score_tensor.csv")

In [10]:
pass_arrived_score.head(150)

,group,gameId,playId,defense_nflId,offense_nflId,playId2,defenseId,offenseId,defense_s_x,defense_s_y,...,receiver_s_x,receiver_s_y,receiver_distance_from_los,defense_football_x,defense_football_y,receiver_football_x,receiver_football_y,football_s,receiver_football_defender_dist_diff,defender_receiver_football_dist_diff
0,full,2018090600,75,79848,310,1,1,1,3.658502,-4.146837,...,2.749019,-1.454302,9.31,12.08,-11.36,16.21,-17.94,2.05,16.409923,-7.596262
1,full,2018090600,75,79848,2533040,1,1,2,3.658502,-4.146837,...,2.749019,-1.454302,9.31,12.08,-11.36,16.21,-17.94,2.05,16.409923,-7.596262
2,full,2018090600,75,79848,2543583,1,1,3,3.658502,-4.146837,...,2.749019,-1.454302,9.31,12.08,-11.36,16.21,-17.94,2.05,16.409923,-7.596262
3,full,2018090600,75,79848,2555415,1,1,4,3.658502,-4.146837,...,2.749019,-1.454302,9.31,12.08,-11.36,16.21,-17.94,2.05,16.409923,-7.596262
4,full,2018090600,75,79848,2559033,1,1,5,3.658502,-4.146837,...,2.749019,-1.454302,9.31,12.08,-11.36,16.21,-17.94,2.05,16.409923,-7.596262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,full,2018090600,146,0,0,2,4,6,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000
146,full,2018090600,146,0,0,2,4,7,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000
147,full,2018090600,146,0,0,2,4,8,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000
148,full,2018090600,146,0,0,2,4,9,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000


In [11]:
pass_arrived_score.columns

Index(['group', 'gameId', 'playId', 'defense_nflId', 'offense_nflId',
       'playId2', 'defenseId', 'offenseId', 'defense_s_x', 'defense_s_y',
       'defense_receiver_x', 'defense_receiver_y', 'defense_receiver_s_x',
       'defense_receiver_s_y', 'defense_offense_x', 'defense_offense_y',
       'defense_offense_s_x', 'defense_offense_s_y', 'receiver_s_x',
       'receiver_s_y', 'receiver_distance_from_los', 'defense_football_x',
       'defense_football_y', 'receiver_football_x', 'receiver_football_y',
       'football_s', 'receiver_football_defender_dist_diff',
       'defender_receiver_football_dist_diff'],
      dtype='object')

# Scoring Classification Data

In [26]:
pass_arrived_score2= pass_arrived_score.drop_duplicates(subset=['gameId', 'playId', 'targetNflId',
                                                                'frameId','defenseId'], keep='first')
pass_arrived_score2 = pass_arrived_score2.drop(['offense_nflId','offenseId','defense_offense_x',\
                                              'defense_offense_y', 'defense_offense_s_x', 'defense_offense_s_y'],
                                            axis = 1)


def create_x(data, nfeatures):
    # creating a numpy tensor to hold the play data
    np_mat = np.array(data.drop(['group', 'gameId', 'playId','targetNflId', 'frameId','defense_nflId'],
                               axis = 1))
    
    # setting parameters
    ngames = np.max(np.array(data['playId2']))
    nDef = 11
    
    # initializing the tensor
    np_tensor = np.zeros((ngames, nDef, nfeatures))
    
    for row in range(len(np_mat)):
        np_tensor[int(np_mat[row][0]) - 1][int(np_mat[row][1]) - 1] = np_mat[row][2:]
    
    np_tensor = np_tensor.astype('float32')
    
    return(np_tensor)

X_score = create_x(data = pass_arrived_score2,
                  nfeatures = 16)

score_preds = model_classification.predict(X_score)

full_preds_df = pd.DataFrame(score_preds, columns = ['IN_prob', 'I_prob', 'C_prob'])

full_preds_df2 = pd.concat([pass_arrived_score[["gameId", "playId", "targetNflId", "frameId"]].drop_duplicates().reset_index(), full_preds_df], axis = 1)
full_preds_df2 = full_preds_df2[full_preds_df2["gameId"] < 10000000000]

full_preds_df2 = full_preds_df2.drop(['index'], axis = 1)

In [31]:
full_preds_df2.to_csv("~/Desktop/CoverageNet/src/03_coverageNet/03_score_tracking/intermediates/intermediate_pass_attempt_classification_probs.csv", 
                        index = False)

# Saving Pass Caught Yards Data

In [12]:
def create_x(data, nfeatures):
    # creating a numpy tensor to hold the play data
    np_mat = np.array(data.drop(['group', 'gameId', 'playId','targetNflId', 'frameId',
                                 'defense_nflId', 'offense_nflId'],
                               axis = 1))
    
    # setting parameters
    ngames = np.max(np.array(data['playId2']))
    nDef = 11
    nOff = 10
    
    # initializing the tensor
    np_tensor = np.zeros((ngames, nDef, nOff, nfeatures))
    
    for row in range(len(np_mat)):
        np_tensor[int(np_mat[row][0]) - 1][int(np_mat[row][1]) - 1][int(np_mat[row][2]) - 1] = np_mat[row][3:]
    
    np_tensor = np_tensor.astype('float32')
    
    return(np_tensor)

X_score = create_x(data = pass_arrived_score,
                  nfeatures = 20)

# Saving Predictions

score_preds = model_caught_yards.predict(X_score)


full_preds_df = pd.DataFrame(score_preds, columns = [str(int(x - 99)) for x in np.linspace(0,198, 199)])

full_preds_df2 = pd.concat([pass_arrived_score[["gameId","playId","targetNflId", "frameId"]].drop_duplicates().reset_index(), full_preds_df], axis = 1)
full_preds_df2 = full_preds_df2[full_preds_df2["gameId"] < 10000000000]
full_preds_df2 = full_preds_df2.drop(['index'], axis = 1)


full_preds_df3 = pd.melt(full_preds_df2, 
        id_vars=['gameId', 'playId', "targetNflId", "frameId"], 
        value_vars=[str(int(x - 99)) for x in np.linspace(0,198, 199)],
        var_name='offensePlayResult', 
        value_name='probability')

full_preds_df3["offensePlayResult"] = [int(x) for x in np.array(full_preds_df3["offensePlayResult"])]
full_preds_df3 = full_preds_df3.sort_values(['gameId','playId',"targetNflId", "frameId",'offensePlayResult'])

KeyError: Index(['frameId', 'targetNflId'], dtype='object')

In [49]:
full_preds_df3.to_csv("~/Desktop/CoverageNet/src/03_coverageNet/03_score_tracking/intermediates/intermediate_pass_caught_yards_probs.csv", 
                        index = False)

# Saving Interceptions Data

In [50]:
# Formatting the Data
pass_arrived_score2= pass_arrived_score.drop_duplicates(subset=['gameId', 'playId', 'targetNflId',
                                                                'frameId','defenseId'], keep='first')

def create_x(data, nfeatures):
    # creating a numpy tensor to hold the play data
    np_mat = np.array(data.drop(['group', 'gameId', 'playId','targetNflId', 'frameId',
                                 'defense_nflId', 'offense_nflId'],
                               axis = 1))
    
    # setting parameters
    ngames = np.max(np.array(data['playId2']))
    nDef = 11
    nOff = 10
    
    # initializing the tensor
    np_tensor = np.zeros((ngames, nDef, nOff, nfeatures))
    
    for row in range(len(np_mat)):
        np_tensor[int(np_mat[row][0]) - 1][int(np_mat[row][1]) - 1][int(np_mat[row][2]) - 1] = np_mat[row][3:]
    
    np_tensor = np_tensor.astype('float32')
    
    return(np_tensor)

X_score = create_x(data = pass_arrived_score2,
                  nfeatures = 20)

# Saving Predictions

score_preds = model_interception_yards.predict(X_score)


full_preds_df = pd.DataFrame(score_preds, columns = [str(int(x - 99)) for x in np.linspace(0,198, 199)])

full_preds_df2 = pd.concat([pass_arrived_score[["gameId","playId","targetNflId", "frameId"]].drop_duplicates().reset_index(), full_preds_df], axis = 1)
full_preds_df2 = full_preds_df2[full_preds_df2["gameId"] < 10000000000]
full_preds_df2 = full_preds_df2.drop(['index'], axis = 1)


full_preds_df3 = pd.melt(full_preds_df2, 
        id_vars=['gameId', 'playId', "targetNflId", "frameId"], 
        value_vars=[str(int(x - 99)) for x in np.linspace(0,198, 199)],
        var_name='offensePlayResult', 
        value_name='probability')

full_preds_df3["offensePlayResult"] = [int(x) for x in np.array(full_preds_df3["offensePlayResult"])]
full_preds_df3 = full_preds_df3.sort_values(['gameId','playId',"targetNflId", "frameId",'offensePlayResult'])

In [52]:
full_preds_df3.to_csv("~/Desktop/CoverageNet/src/03_coverageNet/03_score_tracking/intermediates/intermediate_interceptions_yards_probs.csv", 
                        index = False)